In [1]:
%cd /home/ubuntu/statefarm
from theano.sandbox import cuda
cuda.use('gpu0')
%matplotlib inline
from __future__ import print_function, division
#path = "/home/ubuntu/statefarm/"
path = '/home/ubuntu/statefarm/sample/'
import utils; reload(utils)
from utils import *
from IPython.display import FileLink

/home/ubuntu/statefarm


Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


# First, make the validation set with *different* drivers

In [ ]:
%%bash
cut -f 1 -d ',' driver_imgs_list.csv | grep -v subject | uniq -c
lines=$(expr `wc -l driver_imgs_list.csv | cut -f 1 -d ' '` - 1)
echo "Got ${lines} pics"

fastai's statefarm has 3478 pics in validation set and 18946 in training, so let's get something close to that

In [ ]:
import csv
import os
to_get = set(['p081','p075', 'p072', 'p066', 'p064'])
with open('driver_imgs_list.csv') as f:
    next(f)
    for line in csv.reader(f):
        if line[0] in to_get:
            if os.path.exists('train/%s/%s' %(line[1], line[2])):
                os.popen('mv train/%s/%s valid/%s/%s'%(line[1], line[2], line[1], line[2]))

import glob
print('Training has', len(glob.glob('train/*/*jpg')))
print('Validation has', len(glob.glob('valid/*/*jpg')))

# Now let's make a sample set

In [ ]:
%%bash
mkdir /home/ubuntu/statefarm/sample
mkdir /home/ubuntu/statefarm/sample/train
for i in {0..9}; do mkdir /home/ubuntu/statefarm/sample/train/c${i}; done
mkdir /home/ubuntu/statefarm/sample/valid
for i in {0..9}; do mkdir /home/ubuntu/statefarm/sample/valid/c${i}; done
mkdir /home/ubuntu/statefarm/sample/test
for i in {0..9}; do mkdir /home/ubuntu/statefarm/sample/test/c${i}; done

In [ ]:
from shutil import copyfile
%cd /home/ubuntu/statefarm/train
g = glob.glob('c?/*.jpg')
shuf = np.random.permutation(g)
print(shuf[:5])
for i in range(1500): copyfile(shuf[i], '../sample/train/' + shuf[i])

In [ ]:
%cd ../valid
g = glob.glob('c?/*.jpg')
shuf = np.random.permutation(g)
for i in range(1000): copyfile(shuf[i], '../sample/valid/' + shuf[i])

In [ ]:
%%bash
find /home/ubuntu/statefarm/sample/valid -type f | wc -l
find /home/ubuntu/statefarm/sample/train -type f | wc -l

# So we made a sample set now, hooray, now starts the actual work

In [2]:
batch_size = 64

gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)

trn_batches = get_batches(path+'train', gen_t, batch_size=batch_size)
val_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=False)

Found 1500 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.


In [3]:
from vgg16bn import Vgg16BN
model = vgg_ft_bn(10)

THE FOLLOWING TAKES FOREVER - TAKES MOST OF MEMORY

In [4]:
def get_data(path, target_size=(224,224)):
    batches = get_batches(path, shuffle=False, batch_size=1, class_mode=None, target_size=target_size)
    return np.concatenate([batches.next() for i in range(batches.nb_sample)])

#trn = np.concatenate([trn_batches.next() for i in range(trn_batches.nb_sample)])
#val = np.concatenate([val_batches.next() for i in range(val_batches.nb_sample)])

In [5]:
model.compile(optimizer=Adam(1e-3),
       loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
model.fit_generator(trn_batches, trn_batches.N, nb_epoch=10, validation_data=val_batches, 
                 nb_val_samples=val_batches.N)

Epoch 1/10
1500/1500 [==============================] - 62s - loss: 5.1128 - acc: 0.1787 - val_loss: 3.3818 - val_acc: 0.1390
Epoch 2/10
1500/1500 [==============================] - 62s - loss: 3.7676 - acc: 0.3100 - val_loss: 3.7121 - val_acc: 0.1730
Epoch 3/10
1500/1500 [==============================] - 62s - loss: 3.3112 - acc: 0.3700 - val_loss: 3.6866 - val_acc: 0.2080
Epoch 4/10
1500/1500 [==============================] - 62s - loss: 2.7351 - acc: 0.4447 - val_loss: 3.2837 - val_acc: 0.2570
Epoch 5/10
1500/1500 [==============================] - 62s - loss: 2.6851 - acc: 0.4767 - val_loss: 2.9753 - val_acc: 0.2820
Epoch 6/10
1500/1500 [==============================] - 62s - loss: 2.5379 - acc: 0.4780 - val_loss: 3.3070 - val_acc: 0.2740
Epoch 7/10
1500/1500 [==============================] - 62s - loss: 2.4977 - acc: 0.5027 - val_loss: 3.0505 - val_acc: 0.3310
Epoch 8/10
1500/1500 [==============================] - 62s - loss: 2.3929 - acc: 0.5247 - val_loss: 3.2969 - val_acc:

In [18]:
model.optimizer.lr = 1e-5
model.fit_generator(trn_batches, trn_batches.N, nb_epoch=5, validation_data=val_batches, 
                 nb_val_samples=val_batches.N)

Epoch 1/5
1500/1500 [==============================] - 62s - loss: 1.9820 - acc: 0.5620 - val_loss: 3.0467 - val_acc: 0.3660
Epoch 2/5
1500/1500 [==============================] - 62s - loss: 2.0741 - acc: 0.5700 - val_loss: 3.0053 - val_acc: 0.3850
Epoch 3/5
1500/1500 [==============================] - 62s - loss: 1.8943 - acc: 0.5820 - val_loss: 3.2091 - val_acc: 0.3600
Epoch 4/5
1500/1500 [==============================] - 62s - loss: 1.8415 - acc: 0.5953 - val_loss: 2.8547 - val_acc: 0.3820
Epoch 5/5
1500/1500 [==============================] - 62s - loss: 1.8956 - acc: 0.5960 - val_loss: 3.1333 - val_acc: 0.3920


In [19]:
conv_layers,fc_layers = split_at(model, Convolution2D)

In [20]:
conv_model = Sequential(conv_layers)

In [21]:
conv_feat = conv_model.predict_generator(trn_batches, trn_batches.N)
conv_val_feat = conv_model.predict_generator(val_batches, val_batches.N)


In [22]:
%cd /home/ubuntu/statefarm/sample
%mkdir results

save_array(path+'results/conv_val_feat.dat', conv_val_feat)
save_array(path+'results/conv_feat.dat', conv_feat)
print(type(conv_feat))

conv_feat = load_array(path+'results/conv_feat.dat')
conv_val_feat = load_array(path+'results/conv_val_feat.dat')
print(type(conv_feat))

/home/ubuntu/statefarm/sample
mkdir: cannot create directory ‘results’: File exists
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>


In [23]:
def get_bn_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        BatchNormalization(axis=1),
        Dropout(p),
        Flatten(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(10, activation='softmax')
    ]

In [24]:
p = 0.5

In [25]:
(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes(path)

Found 1500 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
Found 0 images belonging to 10 classes.


In [26]:
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


In [27]:
bn_model.fit(conv_feat, trn_labels, batch_size=batch_size, nb_epoch=1, 
             validation_data=(conv_val_feat, val_labels))

Train on 1500 samples, validate on 1000 samples
Epoch 1/1
1500/1500 [==============================] - 1s - loss: 3.7744 - acc: 0.0967 - val_loss: 6.2962 - val_acc: 0.1320


In [28]:
bn_model.optimizer.lr = 1e-7
bn_model.fit(conv_feat, trn_labels, batch_size=batch_size, nb_epoch=7, 
             validation_data=(conv_val_feat, val_labels))

Train on 1500 samples, validate on 1000 samples
Epoch 1/7
1500/1500 [==============================] - 1s - loss: 2.6958 - acc: 0.2060 - val_loss: 4.5711 - val_acc: 0.1210
Epoch 2/7
1500/1500 [==============================] - 1s - loss: 2.2441 - acc: 0.2960 - val_loss: 3.3890 - val_acc: 0.1230
Epoch 3/7
1500/1500 [==============================] - 1s - loss: 1.8514 - acc: 0.3920 - val_loss: 3.2669 - val_acc: 0.1160
Epoch 4/7
1500/1500 [==============================] - 1s - loss: 1.4936 - acc: 0.5133 - val_loss: 2.9854 - val_acc: 0.1230
Epoch 5/7
1500/1500 [==============================] - 1s - loss: 1.1830 - acc: 0.5993 - val_loss: 3.0020 - val_acc: 0.0950
Epoch 6/7
1500/1500 [==============================] - 1s - loss: 0.9200 - acc: 0.6867 - val_loss: 3.1331 - val_acc: 0.1010
Epoch 7/7
1500/1500 [==============================] - 1s - loss: 0.6866 - acc: 0.7593 - val_loss: 3.1249 - val_acc: 0.0720


# The validation accuracy is 10%, which is essentially random for for 10 classes